In [ ]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import random
import time
from IPython.display import display, clear_output
from tqdm.auto import tqdm
import os, shutil, subprocess

# Parameters
init_proba = 0.2
grid_shape = [108, 192]
nb_steps = 200
time_step = 0.1
video_size_factor = 5
video_frame_rate = 5
tmp_dir = 'img'
interactive_visual = True
video_encoding = True


# *********************************************************


def init_random_grid(grid, init_proba):
  for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
      grid[i, j] = float(random.uniform(0.0, 1.0) < init_proba)

def init_grid(grid):
  init_random_grid(grid, init_proba)

def update_grid(grid):
  new_grid = np.copy(grid)
  for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
      new_grid[i,j] = grid[i,j]
      nb_living = 0
      for k in range(i-1, i+2):
        for l in range(j-1, j+2):
          if k == i and l == j:
            continue
          k = k % grid_shape[0]
          l = l % grid_shape[1]
          nb_living += grid[k,l]
      if grid[i,j] == 1.0 and (nb_living < 2 or nb_living > 3):
        new_grid[i,j] = 0.0
      if grid[i,j] == 0.0 and nb_living == 3:
        new_grid[i,j] = 1.0
  return new_grid


grid = np.zeros(grid_shape)
init_grid(grid)

if video_encoding:
  shutil.rmtree(tmp_dir, ignore_errors=True)
  os.mkdir(tmp_dir)

if interactive_visual:
  plt.ion()
  figure, ax = plt.subplots(figsize=(10, 8))
  img = ax.imshow(grid, cmap='viridis')
  ax.axis('off')
  ax.set_frame_on(False)

counter = 0
for step in range(nb_steps) if interactive_visual else tqdm(range(nb_steps)):
  if interactive_visual:
    img.set_array(grid)
    display(figure)
    clear_output(wait=True)
    time.sleep(time_step)
  if video_encoding:
    filename = f"img/im_{counter:03d}.png"
    plt.imsave(filename, grid)
  grid = update_grid(grid)
  counter += 1

if video_encoding:
  cmd = f"ffmpeg -y -framerate {video_frame_rate} -pattern_type glob -i '{tmp_dir}/*.png' -vf scale={grid_shape[0]*video_size_factor}:{grid_shape[1]*video_size_factor}:flags=neighbor -c:v libx264 -preset slow -crf 17 -pix_fmt yuv420p -movflags +faststart out.mp4"
  res = subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
  shutil.rmtree(tmp_dir, ignore_errors=True)
